# Data wrangling process

__Created by Jochen Teschke__<br>
__Linkedin:__  https://www.linkedin.com/in/jochen-teschke/<br>__Tableau public:__ https://public.tableau.com/app/profile/jochen.teschke

__Data Source:__ The dataset "Liste der Ladesäulen (xlsx, Ladesaeulenregister_BNetzA_2025-09-24.xlsx, 15 MB)" 
was obtained from the "bundesnetzagentur.de"(www.bundesnetzagentur.de). The dataset is licensed under the Creative Commons Attribution 4.0 International (CC BY 4.0) license.<br>__Licence:__ https://creativecommons.org/licenses/by/4.0/<br>__Modifications:__ The original dataset was modified during data preparation. Redundant columns were removed, missing values cleaned where necessary, spelling errors corrected, duplicates deleted, and invalid characters removed.

## Table of Contents
- [Import of python libraries](#Library-import)<br>
- [Used functions](#Creating-two-user-defined-functions)<br>
- [Used dataset](#Loading-the-dataset)<br>
- [Data cleaning](#Data-cleaning)<br>
    - [Deleting columns](#Deleting-columns)<br>
    - [Handling NaN values](#Handling-NaN-values)<br>
    - [Handling duplicates](#Handling-duplicates)<br>
    - [Deleting incorrect leading characters](#Deleting-incorrect-leading-characters)<br>
    - [Correcting spelling errors](#Correcting-spelling-errors)<br>
- [Final evaluation](#Validation-of-the-dataset)<br>
- [Loading the dataset into databases](#Saving-the-dataset)<br>

## Library import

In [1]:
# import of pandas and sqlalchemy
import pandas as pd
import sqlalchemy as db

## Creating two user-defined functions
- the first function deletes duplicate values in cells
- the second function converts column titles to a list and count values in each column using a for loop

In [2]:
def del_cell_duplicates(dataset,column):
    # create a copy of the DataFrame and assign it to a local variable
    df = dataset.copy()

    # cell entries of "column" are converted to list items
    df = df.assign(new_split=df[column].astype(str).str.split(";"))

    # each list item becomes a separate row entry for the same row index
    df = df.explode("new_split")

    # whitespaces are deleted
    df["new_split"] = df["new_split"].astype(str).str.strip()

    # completely duplicate rows are deleted
    df = df.drop_duplicates()

    # everything will be put together and assigned to the original DataFrame
    # .join is a build-in function of python-strings and is applied to a delimiter
    dataset[column+"_clean"] = df.groupby(df.index)["new_split"].agg(lambda x: "; ".join(sorted(x)))

    return dataset

In [3]:
def count_values(dataset):
    # creating a list with all column titles
    column_titles = dataset.columns.tolist()

    # counting values of each column
    for i in column_titles:
        print(dataset.value_counts(i, ascending=False))
        print()

## Loading the dataset
- The dataset is loaded from the internet via URL
- Checking the dataset structure

In [4]:
file_path = "https://data.bundesnetzagentur.de/Bundesnetzagentur/DE/Fachthemen/ElektrizitaetundGas/E-Mobilitaet/Ladesaeulenregister_BNetzA_2025-09-24.xlsx"

In [5]:
dataset = pd.read_excel(file_path,skiprows=10)

In [6]:
dataset.head()

,Ladeeinrichtungs-ID,Betreiber,Anzeigename (Karte),Status,Art der Ladeeinrichtung,Anzahl Ladepunkte,Nennleistung Ladeeinrichtung [kW],Inbetriebnahmedatum,Straße,Hausnummer,...,EVSE-ID4,Public Key4,Steckertypen5,Nennleistung Stecker5,EVSE-ID5,Public Key5,Steckertypen6,Nennleistung Stecker6,EVSE-ID6,Public Key6
0,1010338,Albwerk Elektro- und Kommunikationstechnik GmbH,Albwerk Elektro- und Kommunikationstechnik GmbH,In Betrieb,Normalladeeinrichtung,2,22.0,2020-01-11,Ennabeurer Weg,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1063962,smopi® - Multi Chargepoint Solution GmbH,smopi,In Betrieb,Normalladeeinrichtung,4,22.0,2022-11-23,Albstraße,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1063963,smopi® - Multi Chargepoint Solution GmbH,smopi,In Betrieb,Normalladeeinrichtung,4,22.0,2022-11-23,Albstraße,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1063964,smopi® - Multi Chargepoint Solution GmbH,smopi,In Betrieb,Normalladeeinrichtung,2,22.0,2022-11-23,Albstraße,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1010329,Albwerk Elektro- und Kommunikationstechnik GmbH,Albwerk Elektro- und Kommunikationstechnik GmbH,In Betrieb,Normalladeeinrichtung,2,22.0,2019-01-07,Parkplatz Campingplatz,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95565 entries, 0 to 95564
Data columns (total 47 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Ladeeinrichtungs-ID                95565 non-null  int64         
 1   Betreiber                          95565 non-null  object        
 2   Anzeigename (Karte)                41438 non-null  object        
 3   Status                             95565 non-null  object        
 4   Art der Ladeeinrichtung            95565 non-null  object        
 5   Anzahl Ladepunkte                  95565 non-null  int64         
 6   Nennleistung Ladeeinrichtung [kW]  95565 non-null  float64       
 7   Inbetriebnahmedatum                95565 non-null  datetime64[ns]
 8   Straße                             95565 non-null  object        
 9   Hausnummer                         95565 non-null  object        
 10  Adresszusatz                      

## Data cleaning
- Deleting unnecessary columns
- Handling NaN values, duplicate rows, and duplicates within cells
- Deleting incorrect leading characters
- Correcting spelling errors

#### Deleting columns

In [8]:
# deleting columns which are not required for further analysis
dataset = dataset.drop(columns=["Anzeigename (Karte)", "Adresszusatz",
                      "Standortbezeichnung","EVSE-ID1","EVSE-ID2","EVSE-ID3","EVSE-ID4",
                      "EVSE-ID5","EVSE-ID6", "Public Key1", "Public Key2", "Public Key3",
                      "Public Key4", "Public Key5", "Public Key6",
                      "Nennleistung Stecker1", "Nennleistung Stecker2", "Nennleistung Stecker3",
                      "Nennleistung Stecker4","Nennleistung Stecker5","Nennleistung Stecker6"])

#### Handling NaN values

In [9]:
# check for NaN-Values
dataset.isnull().sum()

Ladeeinrichtungs-ID                      0
Betreiber                                0
Status                                   0
Art der Ladeeinrichtung                  0
Anzahl Ladepunkte                        0
Nennleistung Ladeeinrichtung [kW]        0
Inbetriebnahmedatum                      0
Straße                                   0
Hausnummer                               0
Postleitzahl                             0
Ort                                      0
Kreis/kreisfreie Stadt                   0
Bundesland                               0
Breitengrad                              0
Längengrad                               0
Informationen zum Parkraum           63629
Bezahlsysteme                        13863
Öffnungszeiten                           0
Öffnungszeiten: Wochentage           73448
Öffnungszeiten: Tageszeiten          73448
Steckertypen1                            0
Steckertypen2                        25718
Steckertypen3                        90054
Steckertype

In [10]:
# all NaN values will be replaced by "Keine Angabe"
dataset = dataset.fillna("Keine Angabe")

In [11]:
# check for NaN-Values after replacing them
dataset.isnull().sum()

Ladeeinrichtungs-ID                  0
Betreiber                            0
Status                               0
Art der Ladeeinrichtung              0
Anzahl Ladepunkte                    0
Nennleistung Ladeeinrichtung [kW]    0
Inbetriebnahmedatum                  0
Straße                               0
Hausnummer                           0
Postleitzahl                         0
Ort                                  0
Kreis/kreisfreie Stadt               0
Bundesland                           0
Breitengrad                          0
Längengrad                           0
Informationen zum Parkraum           0
Bezahlsysteme                        0
Öffnungszeiten                       0
Öffnungszeiten: Wochentage           0
Öffnungszeiten: Tageszeiten          0
Steckertypen1                        0
Steckertypen2                        0
Steckertypen3                        0
Steckertypen4                        0
Steckertypen5                        0
Steckertypen6            

#### Handling duplicates

In [12]:
# check for duplicate IDs
print(dataset.duplicated(subset=["Ladeeinrichtungs-ID"]).sum())

0


In [13]:
# check for duplicate rows
print(dataset.duplicated().sum())

0


__Notice__
- Some columns contain duplicate values within the cells
- The value_counts() function helps to investigate this problem
- For clarity, only one column is shown here as an example

In [14]:
# check for duplicate values within cells
dataset["Steckertypen2"].value_counts()

Steckertypen2
AC Typ 2 Steckdose                                                                                  45372
Keine Angabe                                                                                        25718
DC Fahrzeugkupplung Typ Combo 2 (CCS)                                                               15561
AC Typ 2 Fahrzeugkupplung                                                                            3830
AC Typ 2 Steckdose; AC Schuko                                                                        2172
DC Fahrzeugkupplung Typ Combo 2 (CCS); DC CHAdeMO                                                    1563
AC Typ 2 Steckdose; AC Typ 2 Fahrzeugkupplung                                                         914
AC Typ 2 Fahrzeugkupplung; DC Fahrzeugkupplung Typ Combo 2 (CCS)                                      122
AC Typ 2 Steckdose; AC Typ 2 Steckdose                                                                 57
AC Typ 2 Steckdose; DC Fahrzeugk

In [15]:
# calling the user-defined function del_cell_duplicates to delete duplicates within cells of specific columns
dataset = del_cell_duplicates(dataset,"Bezahlsysteme")
dataset = del_cell_duplicates(dataset,"Steckertypen1")
dataset = del_cell_duplicates(dataset,"Steckertypen2")
dataset = del_cell_duplicates(dataset,"Steckertypen3")
dataset = del_cell_duplicates(dataset,"Steckertypen4")
dataset = del_cell_duplicates(dataset,"Steckertypen5")
dataset = del_cell_duplicates(dataset,"Steckertypen6")

#### Deleting incorrect leading characters

In [16]:
# a created list with each unique values shows which leading characters should be deleted from column "Hausnummer"
unique = pd.unique(dataset["Hausnummer"]).tolist()
unique.sort()
print(unique[:40],"...",unique[-40:])

[' 102', ' 2', ' 62', ' 94', '"0"', '-', '.', '0', '00', '01', '05', '0A', '0a', '0b', '0c', '1', '1 ', '1 - 3', '1 - 4', '1 - 5', '1 - 7', '1 A', '1 S', '1 Z', '1 ZZ', '1 ZZZ', '1 a', '1 b', '1 ggü.', '1 p', '1 z', '1 zzz', '1+2', '1+3', '1,2,6', '1-', '1- 3', '1-10', '1-11', '1-12'] ... ['bei 29', 'bei 2a', 'bei 3', 'bei 34', 'bei 35', 'bei 4', 'bei 42', 'bei 5', 'bei 6', 'bei Hs. 2', 'bei Nr. 4', 'bei Nr.9', 'd', 'gg. 81', 'ggÜ. 41', 'ggü .76', 'ggü 27', 'ggü 3', 'ggü 34', 'ggü 5', 'ggü 7', 'ggü. 1', 'ggü. 13', 'ggü. 18', 'ggü. 2', 'ggü. 21', 'ggü. 29b', 'ggü. 33', 'ggü. 5', 'ggü. 6', 'ggü. 7', 'ggü. 76', 'ggü. 90', 'ggü. Nr.18', 'ggü.76', 'ggüb. 1', 'xy', 'zw. 70 /76', '\xa01', '´1']


In [17]:
# deleting wrong leading characters
dataset["Hausnummer"] = dataset["Hausnummer"].str.strip(" \"!.´ ")
dataset["Hausnummer"] = dataset["Hausnummer"].replace({"1-":"1",
                                                       "1 ":"1",
                                                       "01":"1",
                                                       "05":"5", 
                                                       "00":"0",
                                                       "0\"":"0"
                                                       })
dataset["Hausnummer"] = dataset["Hausnummer"].replace({r"^\s+1":"1",
                                                       r"^\s+":"-"},
                                                       regex=True)

#### Correcting spelling errors

In [18]:
# the new column "Bezahlsysteme_clean" has leading characters which should be removed
# and the value "Sonstige" should be replaced by "Keine Angabe"
dataset["Bezahlsysteme_clean"] = dataset["Bezahlsysteme_clean"].str.strip("; ")
dataset["Bezahlsysteme_clean"] = dataset["Bezahlsysteme_clean"].replace("Sonstige", "Keine Angabe")

In [19]:
# in column "Informationen zum Parkraum" are identical values
# which are treated as two different values
# therefore the spelling of one of theses values will be adjusted
dataset["Informationen zum Parkraum"] = dataset["Informationen zum Parkraum"].replace("keine Beschränkung", "Keine Beschränkung")

## Validation of the dataset
- Checking the structure of the cleaned and transformed dataset
- Final compression

In [20]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95565 entries, 0 to 95564
Data columns (total 33 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Ladeeinrichtungs-ID                95565 non-null  int64         
 1   Betreiber                          95565 non-null  object        
 2   Status                             95565 non-null  object        
 3   Art der Ladeeinrichtung            95565 non-null  object        
 4   Anzahl Ladepunkte                  95565 non-null  int64         
 5   Nennleistung Ladeeinrichtung [kW]  95565 non-null  float64       
 6   Inbetriebnahmedatum                95565 non-null  datetime64[ns]
 7   Straße                             95565 non-null  object        
 8   Hausnummer                         95565 non-null  object        
 9   Postleitzahl                       95565 non-null  int64         
 10  Ort                               

In [21]:
# dropping the columns with duplicates within their cells
dataset = dataset.drop(columns=["Bezahlsysteme", "Steckertypen1", "Steckertypen2",
                                "Steckertypen3", "Steckertypen4", "Steckertypen5",
                                "Steckertypen6"])

In [22]:
# rename the columns with the appendix _clean
dataset = dataset.rename(columns={"Bezahlsysteme_clean": "Bezahlsysteme",
                                  "Steckertypen1_clean": "Steckertypen1",
                                  "Steckertypen2_clean": "Steckertypen2",
                                  "Steckertypen3_clean": "Steckertypen3",
                                  "Steckertypen4_clean": "Steckertypen4",
                                  "Steckertypen5_clean": "Steckertypen5",
                                  "Steckertypen6_clean": "Steckertypen6",
                                  })

In [23]:
# call a function to count the values of each column to perform a final evaluation
count_values(dataset)

Ladeeinrichtungs-ID
1134544    1
1000000    1
1000001    1
1000002    1
1000003    1
          ..
1000017    1
1000016    1
1000015    1
1000014    1
1000013    1
Name: count, Length: 95565, dtype: int64

Betreiber
EnBW mobility+ AG und Co.KG                                      4516
Tesla Germany GmbH                                               3584
E.ON Drive GmbH                                                  3002
EWE Go GmbH                                                      1508
Mercedes-Benz AG                                                 1362
                                                                 ... 
ws IT Service GmbH                                                  1
x07907x - Media and IT-Consulting                                   1
Jenny AG                                                            1
Ferien- und Tagungszentrum Alt Schweriner Werder GmbH & Co.KG       1
Ronald  Messner                                                     1
Name: count, Le

In [24]:
# final infos about the cleaned dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95565 entries, 0 to 95564
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Ladeeinrichtungs-ID                95565 non-null  int64         
 1   Betreiber                          95565 non-null  object        
 2   Status                             95565 non-null  object        
 3   Art der Ladeeinrichtung            95565 non-null  object        
 4   Anzahl Ladepunkte                  95565 non-null  int64         
 5   Nennleistung Ladeeinrichtung [kW]  95565 non-null  float64       
 6   Inbetriebnahmedatum                95565 non-null  datetime64[ns]
 7   Straße                             95565 non-null  object        
 8   Hausnummer                         95565 non-null  object        
 9   Postleitzahl                       95565 non-null  int64         
 10  Ort                               

## Saving the dataset
- Upload to a MySQL and a PostgreSQL database
- Save as an Excel file

In [ ]:
# create engines for MySQL and PostgreSQL with sqlalchemy
engine_mysql = db.create_engine("mysql+mysqlconnector://<username>:<password>@<hostname>:<port>/<database>")
engine_postgresql = db.create_engine("postgresql+psycopg2://<username>:<password>@<hostname>:<port>/<database>")

In [ ]:
# full upload of the clean dataset into databases and saving as an Excel file
# the old dataset will be replaced
dataset.to_sql("<table_name>", con=engine_mysql, if_exists="replace")
dataset.to_sql("<table_name>", con=engine_postgresql, if_exists="replace")
dataset.to_excel(r"....\<filename>.xlsx",sheet_name="<sheet_name>", index=False)